In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [28]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = 'hyYCn_kAngI'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [29]:
topics = [{
	"start": 44,
	"finish": 137
},
{
	"start": 137,
	"finish": 220
},{
	"start": 220,
	"finish": 307
},{
	"start": 307,
	"finish": 398
},{
	"start": 398,
	"finish": 517
},{
	"start": 517,
	"finish": 631
},{
	"start": 631,
	"finish": 830
},{
	"start": 830,
	"finish": 923
},{
	"start": 932,
	"finish": 1208
},{
	"start": 1208,
	"finish": 1242
},{
	"start": 1242,
	"finish": 1360
}]


In [30]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

for topic in topics[:1]:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    print(resp.choices[0].message.content)
    print("====================================")

1. **Topic Name:** Importance of Hands-On Approach Early in a Startup's Lifecycle.
2. **Very Very Small Description:** This is about the importance of actively pursuing goals and manually recruiting customers rather than waiting for growth to happen automatically in the early stages of a startup.
3. **Topic Rewrite:**

Hello everyone, let’s discuss something important. If you've ever been curious about running your own startup, you might have come across an essay called "*Do Things That Don't Scale*" by Paul Graham. Paul Graham is one of the folks who started Y-Combinator, a famous startup program, and this essay talks about how some of the biggest companies we know today, like Airbnb, got their start. 

You might think that having a great product or service is all you need to succeed, but that's just part of the picture. The reality is, no product is ever perfect right from the start, especially not in the case of startups. When you show your product to your first customers, they migh